In [ ]:
# Zoren Martinez 2123873

!wget "https://font.download/dl/font/card-characters.zip"
!curl -L -o font.zip "https://fontesk.com/download/3484/"
!unzip card-characters.zip -d ./
!unzip font.zip -d ./


--2025-07-29 15:01:32--  https://font.download/dl/font/card-characters.zip
Resolving font.download (font.download)... 104.26.7.36, 104.26.6.36, 172.67.69.242, ...
Connecting to font.download (font.download)|104.26.7.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22523 (22K) [application/zip]
Saving to: ‘card-characters.zip’

card-characters.zip 100%[===================>]  22.00K  --.-KB/s    in 0s      

2025-07-29 15:01:33 (127 MB/s) - ‘card-characters.zip’ saved [22523/22523]

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  674k  100  674k    0     0   472k      0  0:00:01  0:00:01 --:--:--  472k
Archive:  card-characters.zip
  inflating: ./CARDC___.TTF          
Archive:  font.zip
   creating: ./Khepri/
  inflating: ./Khepri/Khepri-Textured.otf  
  inflating: ./Khepri/Khepri-Round Rough.otf  
  inflating: ./Khepri/Khepri.otf     
  inflatin

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import os
import random
import numpy as np
import cv2

# ======== CONFIGURATION ========
main_font_path = 'CARDC___.TTF'
ten_font_path = '/content/Khepri/Khepri.otf'

characters = ['10', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'J', 'K', 'Q']
output_dir = 'dataset'
os.makedirs(output_dir, exist_ok=True)

font_size = 75
size = 128
img_size = (size, size)
final_binarization_threshold = 180
n_samples_per_class = 800

# ======== PARAMETERS FOR "10" CHARACTER ========
ten_stretch_factor = 1.4
ten_spacing = 8
ten_thickness_factor = 0.55

# ======== ROTATION ANGLE LIMITS FOR 'A' ========
a_rotation_min = -30
a_rotation_max = 0

# ======== AUGMENTATION FUNCTION ========
def add_augmentations(img, char=None):
    np_img = np.array(img).astype(np.uint8)
    h, w = np_img.shape

    # === Random Scaling ===
    scale = random.uniform(1, 1.2)
    scaled = cv2.resize(np_img, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)

    if scale < 1.0:
        # Padding
        pad_h = (h - scaled.shape[0]) // 2
        pad_w = (w - scaled.shape[1]) // 2
        np_img = cv2.copyMakeBorder(scaled, pad_h, h - scaled.shape[0] - pad_h,
                                    pad_w, w - scaled.shape[1] - pad_w,
                                    borderType=cv2.BORDER_CONSTANT, value=255)
    else:
        # Cropping
        start_y = (scaled.shape[0] - h) // 2
        start_x = (scaled.shape[1] - w) // 2
        np_img = scaled[start_y:start_y + h, start_x:start_x + w]

    # === Random Shift ===
    max_shift_x = int(w * 0.15)
    max_shift_y = int(h * 0.15)
    dx = random.randint(-max_shift_x, max_shift_x)
    dy = random.randint(-max_shift_y, max_shift_y)
    M_shift = np.float32([[1, 0, dx], [0, 1, dy]])
    np_img = cv2.warpAffine(np_img, M_shift, (w, h), borderValue=255)

    # === Perspective Warp ===
    margin = 5
    warp_amount = 5
    src = np.float32([[margin, margin], [w - margin, margin],
                      [margin, h - margin], [w - margin, h - margin]])
    dst = src + np.random.uniform(-warp_amount, warp_amount, src.shape).astype(np.float32)
    M_persp = cv2.getPerspectiveTransform(src, dst)
    np_img = cv2.warpPerspective(np_img, M_persp, (w, h), borderValue=255)

    # === Random Rotation ===
    if char == 'A':
        angle = random.uniform(a_rotation_min, a_rotation_max)
    else:
        angle = random.uniform(-20, 20)
    M_rot = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1)
    np_img = cv2.warpAffine(np_img, M_rot, (w, h), borderValue=255)

    # === Morphological Closing (clean small holes) ===
    closing_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
    np_img = cv2.morphologyEx(np_img, cv2.MORPH_CLOSE, closing_kernel)

    # === Resize Down and Back Up (simulate blur or pixelation) ===
    np_img = cv2.resize(np_img, (56, 56), interpolation=cv2.INTER_AREA)
    np_img = cv2.resize(np_img, (w, h), interpolation=cv2.INTER_NEAREST)

    # === Final Binarization ===
    _, np_img = cv2.threshold(np_img, final_binarization_threshold, 255, cv2.THRESH_BINARY)

    return Image.fromarray(np_img)

# ======== DATASET GENERATION LOOP ========
for char in characters:
    char_dir = os.path.join(output_dir, char)
    os.makedirs(char_dir, exist_ok=True)

    for i in range(n_samples_per_class):
        img_base = Image.new('L', img_size, color=255)
        draw = ImageDraw.Draw(img_base)

        font_size_random = random.randint(int(font_size * 0.85), int(font_size * 1.15))
        font = ImageFont.truetype(main_font_path, font_size_random)

        if char == '10':
            # Special case rendering for '10' using two glyphs: '1' and '0'
            font_10 = ImageFont.truetype(ten_font_path, font_size_random)

            bbox_1 = draw.textbbox((0, 0), '1', font=font_10)
            w1, h1 = bbox_1[2] - bbox_1[0], bbox_1[3] - bbox_1[1]
            img_1 = Image.new('L', (w1, h1), color=255)
            draw_1 = ImageDraw.Draw(img_1)
            draw_1.text((0, 0), '1', font=font_10, fill=0)

            bbox_0 = draw.textbbox((0, 0), '0', font=font_10)
            w0, h0 = bbox_0[2] - bbox_0[0], bbox_0[3] - bbox_0[1]
            img_0 = Image.new('L', (w0, h0), color=255)
            draw_0 = ImageDraw.Draw(img_0)
            draw_0.text((0, 0), '0', font=font_10, fill=0)

            img_1_np = np.array(img_1)
            img_0_np = np.array(img_0)
            stretched_1 = cv2.resize(img_1_np, (int(w1 * ten_thickness_factor), int(h1 * ten_stretch_factor)), interpolation=cv2.INTER_LINEAR)
            stretched_0 = cv2.resize(img_0_np, (int(w0 * ten_thickness_factor), int(h0 * ten_stretch_factor)), interpolation=cv2.INTER_LINEAR)

            combined_width = stretched_1.shape[1] + ten_spacing + stretched_0.shape[1]
            combined_height = max(stretched_1.shape[0], stretched_0.shape[0])
            combined_img = np.full((combined_height, combined_width), 255, dtype=np.uint8)

            offset_y_1 = (combined_height - stretched_1.shape[0]) // 2
            offset_y_0 = (combined_height - stretched_0.shape[0]) // 2
            combined_img[offset_y_1:offset_y_1 + stretched_1.shape[0], 0:stretched_1.shape[1]] = stretched_1
            combined_img[offset_y_0:offset_y_0 + stretched_0.shape[0], stretched_1.shape[1] + ten_spacing:] = stretched_0

            scale = min(img_size[0] / combined_width, img_size[1] / combined_height) * 0.65
            new_w = int(combined_width * scale)
            new_h = int(combined_height * scale)
            combined_img = cv2.resize(combined_img, (new_w, new_h), interpolation=cv2.INTER_AREA)

            final_img = np.full(img_size, 255, dtype=np.uint8)
            offset_y = (img_size[1] - new_h) // 2
            offset_x = (img_size[0] - new_w) // 2
            final_img[offset_y:offset_y + new_h, offset_x:offset_x + new_w] = combined_img

            img_base = Image.fromarray(final_img)

        else:
            # Default rendering for all other characters
            bbox = draw.textbbox((0, 0), char, font=font)
            w_, h_ = bbox[2] - bbox[0], bbox[3] - bbox[1]
            ascender, descender = font.getmetrics()
            y_offset = (ascender - descender) / 2 * 0.5
            pos = ((img_size[0] - w_) / 2, (img_size[1] - h_) / 2 - y_offset)
            draw.text(pos, char, fill=0, font=font)

        img_aug = add_augmentations(img_base, char=char)
        img_aug.save(os.path.join(char_dir, f'{char}_{i}.png'))


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

batch_size = 32
num_epochs = 10
learning_rate = 0.001
dataset_dir = 'dataset'

# Image preprocessing
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.ImageFolder(root=dataset_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

num_classes = len(train_dataset.classes)
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),  # Output: 32x128x128
            nn.ReLU(),
            nn.MaxPool2d(2),                             # Output: 32x64x64

            nn.Conv2d(32, 64, kernel_size=3, padding=1), # Output: 64x64x64
            nn.ReLU(),
            nn.MaxPool2d(2),                             # Output: 64x32x32

            nn.Conv2d(64, 128, kernel_size=3, padding=1),# Output: 128x32x32
            nn.ReLU(),
            nn.MaxPool2d(2),                             # Output: 128x16x16
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),                                # Output: 128x16x16
            nn.Linear(128 * 16 * 16, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN(num_classes=num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    avg_loss = total_loss / len(train_loader)
    accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")

torch.save(model.state_dict(), 'simple_card_classifier_weights.pth')
print("Training completed. Weights saved.")


Epoch [1/10] Loss: 0.7062, Accuracy: 76.63%
Epoch [2/10] Loss: 0.0525, Accuracy: 98.53%
Epoch [3/10] Loss: 0.0294, Accuracy: 99.15%
Epoch [4/10] Loss: 0.0126, Accuracy: 99.64%
Epoch [5/10] Loss: 0.0111, Accuracy: 99.66%
Epoch [6/10] Loss: 0.0203, Accuracy: 99.38%
Epoch [7/10] Loss: 0.0145, Accuracy: 99.64%
Epoch [8/10] Loss: 0.0061, Accuracy: 99.79%
Epoch [9/10] Loss: 0.0110, Accuracy: 99.65%
Epoch [10/10] Loss: 0.0189, Accuracy: 99.50%
Training completed. Weights saved.


In [ ]:
model = SimpleCNN(num_classes=13)
model.load_state_dict(torch.load('simple_card_classifier_weights.pth', map_location='cpu'))
model.eval()

example_input = torch.randn(1, 1, size, size)

traced_model = torch.jit.trace(model, example_input)

traced_model.save("simple_card_classifier_traced.pt")


In [5]:
from google.colab import files
files.download('/content/simple_card_classifier_traced.pt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>